# CS246 - Colab 8
## Bloom Filters

### Setup

In this Colab, we need to install a [bloom_filter](https://github.com/hiway/python-bloom-filter), a Python library which offers an implementation of Bloom filters.  Run the cell below!

In [1]:
!pip install bloom_filter

### Data Loading

From the NLTK (Natural Language ToolKit) library, we import a large list of English dictionary words, commonly used by the very first spell-checking programs in Unix-like operating systems.

In [2]:
import nltk
nltk.download('words')

from nltk.corpus import words
word_list = words.words()
print(f'Dictionary length: {len(word_list)}')
print(word_list[:15])

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Dictionary length: 236736
['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron', 'Aaronic', 'Aaronical', 'Aaronite', 'Aaronitic', 'Aaru']


Then we load another dataset from the NLTK Corpora collection: ```movie_reviews```.

The movie reviews are categorized between *positive* and *negative*, so we construct a list of words (usually called **bag of words**) for each category.

In [3]:
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

neg_reviews = []
pos_reviews = []

for fileid in movie_reviews.fileids('neg'):
  neg_reviews.extend(movie_reviews.words(fileid))
for fileid in movie_reviews.fileids('pos'):
  pos_reviews.extend(movie_reviews.words(fileid))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


### Your task

In this Colab, you will develop a very simplistic spell-checker.  By no means you should think of using it for a real-world use case, but it is an interesting exercise to highlight the strenghts and weaknesses of Bloom Filters!

In [4]:
from bloom_filter import BloomFilter

word_filter = BloomFilter(max_elements=236736)

for word in word_list:
  word_filter.add(word)

word_set = set(word_list)

If you executed the cell above, you now have 3 different variables in your scope:

1.   ```word_list```, a Python list containing the English dictionary (in case insensitive order)
2.   ```word_filter```, a Bloom filter where we have already added all the words in the English dictionary
3.   ```word_set```, a [Python set](https://docs.python.org/3.6/library/stdtypes.html#set-types-set-frozenset) built from the same list of words in the English dictionary

Let's inspect the size of each datastructure using the [getsizeof()](https://docs.python.org/3/library/sys.html#sys.getsizeof) method!



In [5]:
from sys import getsizeof

print(f'Size of word_list (in bytes): {getsizeof(word_list)}')
print(f'Size of word_filter (in bytes): {getsizeof(word_filter)}')
print(f'Size of word_set (in bytes): {getsizeof(word_set)}')

Size of word_list (in bytes): 2055512
Size of word_filter (in bytes): 48
Size of word_set (in bytes): 8388824


You should have noticed how efficient is the Bloom filter in terms of memory footprint!

Now let's find out how fast is the main operation for which we construct Bloom filters: *membership testing*. To do so, we will use the ```%timeit``` IPython magic command, which times the repeated execution of a single Python statement.

In [23]:
%timeit -r 3 "California" in word_list
%timeit -r 3 "California" in word_filter
%timeit -r 3 "California" in word_set

421 µs ± 2.8 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)
16.6 µs ± 2.63 µs per loop (mean ± std. dev. of 3 runs, 100000 loops each)
54.2 ns ± 0.457 ns per loop (mean ± std. dev. of 3 runs, 10000000 loops each)


Notice the performance gap between linear search on a list, multiple hash computations in a Bloom filter, and a single hash computation in a native Python ```Set()```.

We now have all the building blocks required to build our spell-checker, and we understand the performance tradeoffs of each datastructure we chose. Write a function that takes as arguments (1) a list of words, and (2) any of the 3 dictionary datastructures we constructed. The function must return the number of words which **do not appear** in the dictionary.


In [7]:
def f(l, d):
    count = 0
    for word in l:
      if word not in d:
        count += 1
    return count

def error(neg_reviews, num_false_words):
  return 100 * num_false_words / len(neg_reviews)


In [21]:
%timeit -r 1 f(neg_reviews, word_filter)
false_filter = f(neg_reviews, word_filter)
error(neg_reviews, false_filter)

6.46 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


25.797065181509364

In [20]:
%timeit -r 1 f(neg_reviews, word_set)
false_set = f(neg_reviews, word_set)
error(neg_reviews, false_set)

69.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


27.98753021606483

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**! The error rate should have `len(neg_reviews)` in its denominator. As a sanity check, your answers for 4.2 and 4.4 should be between 20 and 30.